In [ ]:
# imports
import os
import yaml
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
import architectures as arches

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

# directories
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

In [ ]:
data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

# Prototyping

In [ ]:
with open('arch_specs.yaml') as archfile:
    arch_dict = yaml.load(archfile)

def run_full_fit(architecture, tune=True, save=True):
    """
    Runs the full fitting experiment, pretraining on later experiments and testing on first.
    Saves data as it goes to avoid eating memory.
    (SORT OF; break this up into two functions)
    """
    
    archname = architecture['name']
    archfunc = getattr(arches, architecture['type'])
    arch = lambda input_var=None: archfunc(input_var, **architecture['kwargs'])
    
    tunekws = {'freeze': True, 'exclude': [-5]}
    
    # start training
    trainer = DefaultTrainer(stopthresh=50, print_interval=20)
    net_list = trainer.train_all(
        architecture=arch, data=data, seed=985227
    )
    
    # save params
    if save:
        for i, n in enumerate(net_list):
            fname = '{} {} split agg fit exp 1-4'.format(archname, i)
            n.save_params(os.path.join(paramsdir, fname))
    if tune:
        tuner = FineTuner(stopthresh=10)
        for i, n in enumerate(net_list):
            for j in range(5):
                fname = '{} {} agg fit exp 1-4 {} tune fit exp 0'.format(archname, i, j)
                params = L.get_all_param_values(n.net)
                net = tuner.train_all(architecture=arch, data=hvhdata, split=j, startparams=params, **tunekws )
                if save:
                    net.save_params(os.path.join(paramsdir, fname))
    
    return None

In [ ]:
with open('arch_specs.yaml') as archfile:
    arch_dict = yaml.load(archfile)
    
architecture = arch_dict['h4']

af = getattr(arches, architecture['type'])
net = af(**architecture['kwargs'])
print('N params:', L.count_params(net))
    
architecture

In [ ]:
run_full_fit(architecture, tune=True) 

## Subject tuning

Usually doesn't work (not enough data even for very simple classifier layers)

In [ ]:
dafiname = os.path.join(datadir, '0 (with groups).csv')
subject_data = [loading.default_loader(dafiname, subject=s) for s in range(40)]
arch = archs[archname]

In [ ]:
print([len(s[0]) for s in subject_data])

In [ ]:
for i in range(5):
    pafiname = '{} {} split agg fit exp 1-4.npz'.format(archname, i)
    prenet = Network(arch)
    prenet.load_params(os.path.join(paramsdir, pafiname))
    params = L.get_all_param_values(prenet.net)
    print('PREFIT {}\n'.format(i))
    
    for s in range(40):
        sdata = subject_data[s]
        num_obs = len(sdata[0])
        bs = num_obs//5
#         if num_obs > 50:
        tuner = FineTuner(stopthresh=10, batchsize=bs)
        print('SUBJECT {}\n'.format(s))
        
        for j in range(5):
            fname = '{} {} agg fit exp 1-4 {} subject {} tune fit exp 0'.format(archname, i, s, j)
            net = tuner.train_all(architecture=arch, data=sdata, split=j, startparams=params, freeze=True)
            net.save_params(os.path.join(paramsdir, fname))

## Data aggregation

doesn't need run more than once

In [ ]:
datafilenames = ['0 (with groups)', '1 (with computer)', '2 (with computer)', '3 (with computer)', '4']
datafilenames = [os.path.join(datadir, fname + '.csv') for fname in datafilenames]
colnames = ['subject', 'color', 'bp', 'wp', 'zet', 'rt']

e0 = pd.read_csv(datafilenames[0], names=colnames+['splitno'])
e1 = pd.read_csv(datafilenames[1], names=colnames)
e2 = pd.read_csv(datafilenames[2], names=colnames)
e3 = pd.read_csv(datafilenames[3], names=colnames+['task', 'taskorder', 'session'])
e4 = pd.read_csv(datafilenames[4], names=colnames+['timecondition'])
Es = [e1, e2, e3, e4]
for i, e in enumerate(Es[1:]):
    e['subject'] = e['subject'] + Es[i-1].loc[Es[i-1]['subject']<1000, 'subject'].max()

A = pd.concat([e[colnames] for e in [e1, e2, e3, e4]])

groups = np.arange(len(A))%5 + 1
np.random.seed(100001)
np.random.shuffle(groups)
A['group'] = groups

A.to_csv(os.path.join(datadir, '1-4.csv'), encoding='ASCII', header=False, index=False)
A.loc[A['subject']<1000, :].to_csv(
    os.path.join(datadir, '1-4 (no computer).csv'), 
    encoding='ASCII', header=False, index=False
)